# 数据增强问答

本笔记本使用一些通用的提示/语言模型来评估使用模型之外的其他数据源的问答系统。例如，这可以用于评估使用您专有数据的问答系统。

## 设置
让我们设置一个例子，用我们最喜欢的例子-国情咨文。

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

In [2]:
from langchain.document_loaders import TextLoader
loader = TextLoader('../../modules/state_of_the_union.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)
qa = RetrievalQA.from_llm(llm=OpenAI(), retriever=docsearch.as_retriever())

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


## 例子
现在我们需要一些示例来评估。我们可以以两种方式做到这一点：

1. 自己硬编码一些示例
2. 使用语言模型自动生成示例

In [3]:
# Hard-coded examples
examples = [
    {
        "query": "What did the president say about Ketanji Brown Jackson",
        "answer": "He praised her legal ability and said he nominated her for the supreme court."
    },
    {
        "query": "What did the president say about Michael Jackson",
        "answer": "Nothing"
    }
]

In [4]:
# Generated examples
from langchain.evaluation.qa import QAGenerateChain
example_gen_chain = QAGenerateChain.from_llm(OpenAI())

In [5]:
new_examples = example_gen_chain.apply_and_parse([{"doc": t} for t in texts[:5]])

In [6]:
new_examples

[{'query': 'According to the document, what did Vladimir Putin miscalculate?',
  'answer': 'He miscalculated that he could roll into Ukraine and the world would roll over.'},
 {'query': 'Who is the Ukrainian Ambassador to the United States?',
  'answer': 'The Ukrainian Ambassador to the United States is here tonight.'},
 {'query': 'How many countries were part of the coalition formed to confront Putin?',
  'answer': '27 members of the European Union, France, Germany, Italy, the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland.'},
 {'query': 'What action is the U.S. Department of Justice taking to target Russian oligarchs?',
  'answer': 'The U.S. Department of Justice is assembling a dedicated task force to go after the crimes of Russian oligarchs and joining with European allies to find and seize their yachts, luxury apartments, and private jets.'},
 {'query': 'How much direct assistance is the United States providing to Ukraine?',
  'answ

In [7]:
# Combine examples
examples += new_examples

## 评估
现在我们有了示例，我们可以使用问答系统评估器来评估我们的问答链。

In [8]:
from langchain.evaluation.qa import QAEvalChain

In [9]:
predictions = qa.apply(examples)

In [10]:
llm = OpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)

In [11]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [12]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

Example 0:
Question: What did the president say about Ketanji Brown Jackson
Real Answer: He praised her legal ability and said he nominated her for the supreme court.
Predicted Answer:  The president said that she is one of the nation's top legal minds, a former top litigator in private practice, a former federal public defender, and from a family of public school educators and police officers. He also said that she is a consensus builder and that she has received a broad range of support from the Fraternal Order of Police to former judges appointed by both Democrats and Republicans.
Predicted Grade:  CORRECT

Example 1:
Question: What did the president say about Michael Jackson
Real Answer: Nothing
Predicted Answer:  The president did not mention Michael Jackson in this speech.
Predicted Grade:  CORRECT

Example 2:
Question: According to the document, what did Vladimir Putin miscalculate?
Real Answer: He miscalculated that he could roll into Ukraine and the world would roll over.
Pred

## 使用其他指标进行评估

除了使用语言模型预测答案是否正确或错误之外，我们还可以使用其他指标来获得更精细的答案质量视图。为此，我们可以使用 [Critique](https://docs.inspiredco.ai/critique/) 库，它允许对生成的文本进行各种指标的简单计算。

首先，您可以从 [Inspired Cognition Dashboard](https://dashboard.inspiredco.ai) 获取 API 密钥并进行一些设置：

```bash
export INSPIREDCO_API_KEY="..."
pip install inspiredco
```

In [13]:
import inspiredco.critique
import os
critique = inspiredco.critique.Critique(api_key=os.environ['INSPIREDCO_API_KEY'])

然后运行以下代码来设置配置并计算 [ROUGE](https://docs.inspiredco.ai/critique/metric_rouge.html)、[chrf](https://docs.inspiredco.ai/critique/metric_chrf.html)、[BERTScore](https://docs.inspiredco.ai/critique/metric_bert_score.html) 和 [UniEval](https://docs.inspiredco.ai/critique/metric_uni_eval.html)（您也可以选择 [其他指标](https://docs.inspiredco.ai/critique/metrics.html)）：

In [14]:
metrics = {
    "rouge": {
        "metric": "rouge",
        "config": {"variety": "rouge_l"},
    },
    "chrf": {
        "metric": "chrf",
        "config": {},
    },
    "bert_score": {
        "metric": "bert_score",
        "config": {"model": "bert-base-uncased"},
    },
    "uni_eval": {
        "metric": "uni_eval",
        "config": {"task": "summarization", "evaluation_aspect": "relevance"},
    },
}

In [15]:
critique_data = [
    {"target": pred['result'], "references": [pred['answer']]} for pred in predictions
]
eval_results = {
    k: critique.evaluate(dataset=critique_data, metric=v["metric"], config=v["config"])
    for k, v in metrics.items()
}

最后，我们可以打印出结果。我们可以看到，当输出在语义上是正确的，并且当输出与金标准答案非常匹配时，得分总体上更高。

In [16]:
for i, eg in enumerate(examples):
    score_string = ", ".join([f"{k}={v['examples'][i]['value']:.4f}" for k, v in eval_results.items()])
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Scores: " + score_string)
    print()

Example 0:
Question: What did the president say about Ketanji Brown Jackson
Real Answer: He praised her legal ability and said he nominated her for the supreme court.
Predicted Answer:  The president said that she is one of the nation's top legal minds, a former top litigator in private practice, a former federal public defender, and from a family of public school educators and police officers. He also said that she is a consensus builder and that she has received a broad range of support from the Fraternal Order of Police to former judges appointed by both Democrats and Republicans.
Predicted Scores: rouge=0.0941, chrf=0.2001, bert_score=0.5219, uni_eval=0.9043

Example 1:
Question: What did the president say about Michael Jackson
Real Answer: Nothing
Predicted Answer:  The president did not mention Michael Jackson in this speech.
Predicted Scores: rouge=0.0000, chrf=0.1087, bert_score=0.3486, uni_eval=0.7802

Example 2:
Question: According to the document, what did Vladimir Putin mis